# English Premier League
This notebook get the scores for the most recent games in the English Premier League and posts them on Twitter. Instead of simply listing all the scores, it prints "surprises" (teams that were expected to lose but ended up winning) first. Using the betting odds predictions we can see if a team that was expected to lose actually won.

Two future projects I am working on:
- printing "surprising" statistics pere player. For instance, if a player had 20 shots on target, but none went it.
- I would like to automatically call these API's daily but I don't have a way to deploy to the cloud and don't have a server at home to run a simple cron job.

In [7]:
import requests
import json
import os

# load the environment variables
from dotenv import load_dotenv
load_dotenv()

# Bring your packages onto the path
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# import functions from src/mlb_scores.py
from epl_scores import get_epl_scores, get_epl_odds, get_correct_upsets, create_tweet


In [2]:
# get yesterday's date as string
import datetime
yesterday = datetime.date.today() - datetime.timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')

# date = '2022-08-14'
date = yesterday

# Get Scores from EPL on date

In [ ]:
def get_epl_scores(date='2022-08-06'):
    """
    Uses API-sports to get the scores of the matches for the given date. Returns a dictionary of match_id: {home_team, home_score, away_team, away_score}

    """
    # get all the ids of a match for a url
    url = "https://v3.football.api-sports.io/fixtures"

    payload = {'league': 39, 'season': 2022, 'date': date}

    headers = {
        'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY"),
        'x-rapidapi-host': 'v3.football.api-sports.io'
    }

    fixtures_response = json.loads(requests.request(
        "GET", url, headers=headers, params=payload).text)

    assert (fixtures_response['errors'] == [])

    score_match_dict = {}
    for match in fixtures_response['response']:
        # get the fixture id
        fixture = match['fixture']['id']

        # get the team names
        home_team = match['teams']['home']['name']
        away_team = match['teams']['away']['name']

        # get the score
        home_score = match['score']['fulltime']['home']
        away_score = match['score']['fulltime']['away']

        # who won?
        if home_score > away_score:
            winner = 'Home'
        elif home_score < away_score:
            winner = 'Away'
        else:
            winner = 'Draw'

        score_match_dict[fixture] = {'home_team': home_team, 'away_team': away_team,
                                     'home_score': home_score, 'away_score': away_score, 'winner': winner}

    # returns a dictionary of the match id and the teams and score
    return score_match_dict

In [3]:
score_match_dict = get_epl_scores(date)
score_match_dict

{867976: {'home_team': 'Arsenal',
  'away_team': 'Fulham',
  'home_score': 2,
  'away_score': 1,
  'winner': 'Home'},
 867978: {'home_team': 'Brentford',
  'away_team': 'Everton',
  'home_score': 1,
  'away_score': 1,
  'winner': 'Draw'},
 867979: {'home_team': 'Brighton',
  'away_team': 'Leeds',
  'home_score': 1,
  'away_score': 0,
  'winner': 'Home'},
 867980: {'home_team': 'Chelsea',
  'away_team': 'Leicester',
  'home_score': 2,
  'away_score': 1,
  'winner': 'Home'},
 867981: {'home_team': 'Liverpool',
  'away_team': 'Bournemouth',
  'home_score': 9,
  'away_score': 0,
  'winner': 'Home'},
 867982: {'home_team': 'Manchester City',
  'away_team': 'Crystal Palace',
  'home_score': 4,
  'away_score': 2,
  'winner': 'Home'},
 867984: {'home_team': 'Southampton',
  'away_team': 'Manchester United',
  'home_score': 0,
  'away_score': 1,
  'winner': 'Away'}}

# Get the "Odds of Winning" for each game on the date

In [ ]:
def get_epl_odds(date='2022-08-06'):
    url = "https://v3.football.api-sports.io/odds"

    payload = {'league': 39, 'season': 2022, 'date': date}

    headers = {
        'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY"),
        'x-rapidapi-host': 'v3.football.api-sports.io'
    }

    odds_response = json.loads(requests.request(
        "GET", url, headers=headers, params=payload).text)

    odds_match_dict = {}
    for match in odds_response['response']:

        assert (match['bookmakers'][0]['bets'][0]['name'] == 'Match Winner')

        odd_dict = {}
        for value in match['bookmakers'][0]['bets'][0]['values']:
            odd_dict[value['odd']] = value['value']

        pred_win = odd_dict[min(odd_dict.keys())]

        odds_match_dict[match['fixture']['id']] = pred_win

    return odds_match_dict



In [4]:
odds_match_dict = get_epl_odds(date)
odds_match_dict

{867976: 'Home',
 867978: 'Home',
 867979: 'Home',
 867980: 'Home',
 867981: 'Home',
 867982: 'Home',
 867984: 'Away'}

# Print Outputs

In [5]:
# get lists of correct and incorrect predictions
correct_ids = []
upset_ids = []
for match_id in odds_match_dict.keys():
    if odds_match_dict[match_id]!=score_match_dict[match_id]['winner']:
       upset_ids.append(match_id) 
    else:
        correct_ids.append(match_id)

print('#EPL Scores {date}'.format(date=date))
if len(upset_ids)>0:
    print("SURPRISES:")
    for id in upset_ids:
        score_match_dict[id]
        print(str("{home_team}: {home_score} - {away_team}: {away_score}").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score']))
if len(correct_ids)>0:
    print("\nEXPECTED:")
    for id in correct_ids:
        score_match_dict[id]
        print(str("{home_team}: {home_score} - {away_team}: {away_score}").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score']))

#EPL Scores 2022-08-27
SURPRISES:
Brentford: 1 - Everton: 1

EXPECTED:
Arsenal: 2 - Fulham: 1
Brighton: 1 - Leeds: 0
Chelsea: 2 - Leicester: 1
Liverpool: 9 - Bournemouth: 0
Manchester City: 4 - Crystal Palace: 2
Southampton: 0 - Manchester United: 1


# Get Player Stats
**this is still in dev**

In [78]:
def get_epl_scores(date='2022-08-06'):
    """
    Uses API-sports to get the scores of the matches for the given date. Returns a dictionary of match_id: {home_team, home_score, away_team, away_score}

    """
    # get all the ids of a match for a url
    url = "https://v3.football.api-sports.io/fixtures"

    payload = {'league': 39, 'season': 2022, 'date': date}

    headers = {
        'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY"),
        'x-rapidapi-host': 'v3.football.api-sports.io'
    }

    fixtures_response = json.loads(requests.request(
        "GET", url, headers=headers, params=payload).text)

    assert (fixtures_response['errors'] == [])

    score_match_dict = {}
    for match in fixtures_response['response']:
        # get the fixture id
        fixture = match['fixture']['id']

        # get the team names
        home_team = match['teams']['home']['name']
        away_team = match['teams']['away']['name']

        # get the score
        home_score = match['score']['fulltime']['home']
        away_score = match['score']['fulltime']['away']

        # who won?
        if home_score > away_score:
            winner = 'Home'
        elif home_score < away_score:
            winner = 'Away'
        else:
            winner = 'Draw'

        score_match_dict[fixture] = {'home_team': home_team, 'away_team': away_team,
                                     'home_score': home_score, 'away_score': away_score, 'winner': winner}

    # returns a dictionary of the match id and the teams and score
    return score_match_dict

In [80]:
score_match_dict = get_epl_scores()

In [81]:
score_match_dict.keys()

dict_keys([867947, 867948, 867949, 867951, 867952, 867953])

In [90]:
import pandas as pd

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY")
    }

url = "https://v3.football.api-sports.io/fixtures/"

payload = {'league': 39, 'season': 2021}

headers = {
    'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY"),
    'x-rapidapi-host': 'v3.football.api-sports.io'
}

fixture_data = json.loads(requests.request(
        "GET", url, headers=headers, params=payload).text)

In [101]:
list_of_fixtures = pd.json_normalize(fixture_data['response'])['fixture.id'].to_list()

In [100]:
list_of_fixtures

0      710556
1      710557
2      710558
3      710559
4      710560
        ...  
375    710931
376    710932
377    710933
378    710934
379    710935
Name: fixture.id, Length: 380, dtype: int64

In [134]:
import pandas as pd

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': os.environ.get("RAPIDAPI_KEY")
    }

url = "https://v3.football.api-sports.io/fixtures/players"



# meta_player_list = []
for fixture in list_of_fixtures[40:]:
    payload = {'fixture': fixture}
    
    player_data = json.loads(requests.request(
        "GET", url, headers=headers, params=payload).text)

    player_df = pd.json_normalize(player_data['response'],record_path=['players', 'statistics'], meta=[['players','player','name']])[['games.position','shots.total','shots.on', 'goals.total', 'goals.assists','goals.saves','passes.total', 'passes.key', 'passes.accuracy','tackles.total', 'tackles.blocks', 'tackles.interceptions',
        'duels.total', 'duels.won','fouls.drawn', 'fouls.committed', 'cards.yellow',
        'cards.red', 'penalty.won', 'penalty.commited', 'penalty.scored',
        'penalty.missed', 'penalty.saved','offsides','players.player.name']]

    meta_player_list.append(player_df)



KeyError: "None of [Index(['games.position', 'shots.total', 'shots.on', 'goals.total',\n       'goals.assists', 'goals.saves', 'passes.total', 'passes.key',\n       'passes.accuracy', 'tackles.total', 'tackles.blocks',\n       'tackles.interceptions', 'duels.total', 'duels.won', 'fouls.drawn',\n       'fouls.committed', 'cards.yellow', 'cards.red', 'penalty.won',\n       'penalty.commited', 'penalty.scored', 'penalty.missed', 'penalty.saved',\n       'offsides', 'players.player.name'],\n      dtype='object')] are in the [columns]"

In [135]:
meta_player_df = pd.concat(meta_player_list)

In [136]:
meta_player_df

,games.position,shots.total,shots.on,goals.total,goals.assists,goals.saves,passes.total,passes.key,passes.accuracy,tackles.total,tackles.blocks,tackles.interceptions,duels.total,duels.won,fouls.drawn,fouls.committed,cards.yellow,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved,offsides,players.player.name
0,G,NaN,NaN,NaN,NaN,4.0,46.0,NaN,20,NaN,NaN,NaN,1.0,1.0,NaN,NaN,0,0,None,None,0,0,0.0,NaN,David Raya
1,D,NaN,NaN,NaN,NaN,NaN,31.0,1.0,19,1.0,NaN,NaN,4.0,2.0,NaN,NaN,0,0,None,None,0,0,NaN,NaN,Kristoffer Ajer
2,D,NaN,NaN,NaN,NaN,NaN,22.0,NaN,17,NaN,NaN,1.0,1.0,NaN,NaN,NaN,0,0,None,None,0,0,NaN,NaN,Pontus Jansson
3,D,NaN,NaN,NaN,1.0,NaN,35.0,1.0,23,NaN,2.0,3.0,10.0,5.0,1.0,NaN,0,0,None,None,0,0,NaN,NaN,Ethan Pinnock
4,M,3.0,2.0,1.0,NaN,NaN,19.0,1.0,12,7.0,1.0,3.0,8.0,7.0,NaN,NaN,0,0,None,None,0,0,NaN,NaN,Sergi Canós
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,Saman Ghoddos
36,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,Yoane Wissa
37,D,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1,2.0,NaN,NaN,5.0,3.0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,Mads Roerslev Rasmussen
38,M,NaN,NaN,NaN,NaN,NaN,2.0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,Frank Onyeka


In [137]:
meta_player_groupby = meta_player_df.groupby('games.position').quantile(0.75)


/tmp/ipykernel_352053/3826061352.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.quantile is deprecated. In a future version, a TypeError will be raised. Before calling .quantile, select only columns which should be valid for the function.
  meta_player_groupby = meta_player_df.groupby('games.position').quantile(0.75)


In [153]:
x[1][1]

-28.0

In [154]:
interesting_list = []
for x in pd.concat([meta_player_groupby.loc['D'], meta_player_df.iloc[1]], axis=1).dropna().diff(periods=1, axis=1).iterrows():
    if x[1][1]>0:
        interesting_list.append(x.index)

In [155]:
interesting_list

[]

In [ ]:
for player in player_stats:
    if player['games.position']=='D':
        
    elif player['games.position']=='M':
        pass
    elif player['games.position']=='F':
        pass
    else:
        